In [1]:
# set random seed
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Parameters

In [2]:
import pyepo
# generate data
grid = (5,5) # grid size
num_data = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.2 # polynomial degree
e = 0.5 # noise width

# Generate Data

In [3]:
from Data import data_generation
data_gen = data_generation()

#  ****** Data generation process is the same as SPO+ *********
feats, costs = data_gen.generate_Shortest_Path_Data(num_data+num_test, num_feat, grid, deg, e, seed=42)

#  ****** Data generation process is the same as DDR *********
# lower = 0
# upper = 1
# p = 5
# d = 40
# alpha = 1
# mis = 4
# n_epsilon = 1
# W_star = data_gen.generate_truth("",lower, upper, p, d, version = 0) 
# x_test, z_test_ori, c_test, x_train, z_train_ori, c_train, W_star = data_gen.generate_samples("",p, d, num_test, num_data, alpha, W_star, n_epsilon, mis, thres = 10, 
#                         version = 1, x_dist = 'normal', e_dist = 'normal', x_low = 0, x_up = 2, x_mean = 2, x_var = 0.25, bump = 0) 

# Build Model

In [4]:
# build optModel
from pyepo.model.grb import optGrbModel

class shortestPathModel(optGrbModel):

    def __init__(self):
        self.grid = (5,5)
        self.arcs = self._getArcs()
        super().__init__()

    def _getArcs(self):
        """
        A helper method to get list of arcs for grid network

        Returns:
            list: arcs
        """
        arcs = []
        for i in range(self.grid[0]):
            # edges on rows
            for j in range(self.grid[1] - 1):
                v = i * self.grid[1] + j
                arcs.append((v, v + 1))
            # edges in columns
            if i == self.grid[0] - 1:
                continue
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                arcs.append((v, v + self.grid[1]))
        return arcs

    def _getModel(self):
        """
        A method to build Gurobi model

        Returns:
            tuple: optimization model and variables
        """
        import gurobipy as gp
        from gurobipy import GRB
        # ceate a model
        m = gp.Model("shortest path")
        # varibles
        x = m.addVars(self.arcs, name="x")
        # sense
        m.modelSense = GRB.MINIMIZE
        # flow conservation constraints
        for i in range(self.grid[0]):
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                expr = 0
                for e in self.arcs:
                    # flow in
                    if v == e[1]:
                        expr += x[e]
                    # flow out
                    elif v == e[0]:
                        expr -= x[e]
                # source
                if i == 0 and j == 0:
                    m.addConstr(expr == -1)
                # sink
                elif i == self.grid[0] - 1 and j == self.grid[0] - 1:
                    m.addConstr(expr == 1)
                # transition
                else:
                    m.addConstr(expr == 0)
        return m, x

# Prepara training and test data

### SPO+ data generation process

In [5]:
# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test, random_state=42)

In [6]:
optmodel = shortestPathModel()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-25


In [7]:
# get optDataset
dataset_train = pyepo.data.dataset.optDataset(optmodel, x_train, c_train)
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, c_test)

Test
Optimizing for optDataset...


100%|██████████| 100/100 [00:00<00:00, 1999.33it/s]


Test
Optimizing for optDataset...


100%|██████████| 1000/1000 [00:00<00:00, 2190.34it/s]


In [8]:
# set data loader
from torch.utils.data import DataLoader
batch_size = 20
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

# Linear regression

In [9]:
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, (grid[0]-1)*grid[1]+(grid[1]-1)*grid[0])

    def forward(self, x):
        out = self.linear(x)
        return out

## Initial the predictor

In [10]:
import torch
# init model
reg = LinearRegression()

# Evaluation

In [11]:
import pyepo
regret = pyepo.metric.regret(reg, optmodel, loader_test)

# SPO+

In [12]:
import torch
# init model
reg = LinearRegression()
# cuda
if torch.cuda.is_available():
    reg = reg.cuda()

## Initialize

In [13]:
# init SPO+ loss
spop = pyepo.func.SPOPlus(optmodel, processes=2)

Num of cores: 2


# Training

In [14]:
arcs_arr = optmodel.arcs
def obtain_path(arcs_arr,sol):
    path_arr = []
    for arc_index in range(len(arcs_arr)):
        if sol[arc_index] > 0:
            path_arr.append(arcs_arr[arc_index])
    return path_arr

In [17]:
def getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))
    return arcs

def solve_Shortest_Path(arcs,cost):
    """
    A method to build Gurobi model

    Returns:
        tuple: optimization model and variables
    """
    import gurobipy as gp
    from gurobipy import GRB
    # ceate a model
    m = gp.Model("shortest path")
    m.setParam('OutputFlag', 0)
    # varibles
    x = m.addVars(arcs, name="x")
    # sense
    # m.modelSense = GRB.MINIMIZE
    # flow conservation constraints
    for i in range(grid[0]):
        for j in range(grid[1]):
            v = i * grid[1] + j
            expr = 0
            for e in arcs:
                # flow in
                if v == e[1]:
                    expr += x[e]
                # flow out
                elif v == e[0]:
                    expr -= x[e]
            # source
            if i == 0 and j == 0:
                m.addConstr(expr == -1)
            # sink
            elif i == grid[0] - 1 and j == grid[0] - 1:
                m.addConstr(expr == 1)
            # transition
            else:
                m.addConstr(expr == 0)
    m.setObjective( sum([cost[ind] * x[arcs_arr[ind]] for ind in range(len(arcs_arr))]) , GRB.MINIMIZE)
    m.optimize()
    sol = m.getAttr('x')
    # print("sol = ",sol)
    shortest_path = obtain_path(arcs_arr,sol)
    # print("shortest_path = ",shortest_path)
    return sol

In [17]:
# from pyepo import EPO
def evaluation_SPO(predmodel, optmodel, dataloader):
    """
    A function to evaluate model performance with normalized true regret

    Args:
        predmodel (nn): a regression neural network for cost prediction
        optmodel (optModel): an PyEPO optimization model
        dataloader (DataLoader): Torch dataloader from optDataSet

    Returns:
        float: true regret loss
    """
    # evaluate
    predmodel.eval()
    loss = 0
    optsum = 0
    cost_pred_arr = []
    cost_true_arr = []
    # load data
    for data in dataloader:
        x, c, w, z = data
        # cuda
        if next(predmodel.parameters()).is_cuda:
            x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
        # predict
        with torch.no_grad(): # no grad
            cp = predmodel(x).to("cpu").detach().numpy()
        # print("cp[0] = ",cp[0])
        # solve
        for j in range(cp.shape[0]):
            sol_pred = solve_Shortest_Path(arcs_arr,cp[j])
            cost_pred = np.dot(sol_pred, c[j].to("cpu").detach().numpy())
            cost_pred_arr.append(cost_pred)
            cost_true_arr.append(z[j].item())
    # turn back train mode
    predmodel.train()
    # normalized
    return cost_true_arr,cost_pred_arr

In [18]:
import time
# set adam optimizer
optimizer = torch.optim.Adam(reg.parameters(), lr=1e-2)
# train mode
reg.train()
# init log
cost_true_arr,cost_pred_arr = evaluation_SPO(reg, optmodel, loader_test)

In [19]:
loader_test

In [20]:
num_epochs = 30
# set adam optimizer
optimizer = torch.optim.Adam(reg.parameters(), lr=1e-2)
# train mode
reg.train()
# init log
cost_true_arr,cost_pred_arr = evaluation_SPO(reg, optmodel, loader_test)
print("epoch 0: Average Predict Cost = ", np.mean(cost_pred_arr),", Average True Cost = ",np.mean(cost_true_arr))
# init elpased time
elapsed = 0
for epoch in range(num_epochs):
    # start timing
    tick = time.time()
    # load data
    for i, data in enumerate(loader_train):
        x, c, w, z = data
        # cuda
        if torch.cuda.is_available():
            x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
        # forward pass
        cp = reg(x)
        loss = spop(cp, c, w, z)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # record time
        tock = time.time()
        elapsed += tock - tick
        # log
    cost_true_arr,cost_pred_arr = evaluation_SPO(reg, optmodel, loader_test)
    print("epoch = ",epoch," Average Predict Cost = ", np.mean(cost_pred_arr),", Average True Cost = ",np.mean(cost_true_arr))

epoch 0: Average Predict Cost =  8.29035397285223 , Average True Cost =  6.6683534908294675
epoch =  0  Average Predict Cost =  8.20830439093709 , Average True Cost =  6.6683534908294675
epoch =  1  Average Predict Cost =  8.150820441365243 , Average True Cost =  6.6683534908294675
epoch =  2  Average Predict Cost =  8.054759571045636 , Average True Cost =  6.6683534908294675
epoch =  3  Average Predict Cost =  7.962212584391236 , Average True Cost =  6.6683534908294675
epoch =  4  Average Predict Cost =  7.9320243769586085 , Average True Cost =  6.6683534908294675
epoch =  5  Average Predict Cost =  7.912762609243393 , Average True Cost =  6.6683534908294675
epoch =  6  Average Predict Cost =  7.890208253651857 , Average True Cost =  6.6683534908294675
epoch =  7  Average Predict Cost =  7.878643666177988 , Average True Cost =  6.6683534908294675
epoch =  8  Average Predict Cost =  7.8472349779754875 , Average True Cost =  6.6683534908294675
epoch =  9  Average Predict Cost =  7.81415

# OLS

In [21]:
# from pyepo import EPO
def evaluation_OLS(w0_ols,W_ols, dataloader):

    # evaluate
    cost_pred_arr = []
    # load data
    for data in dataloader:
        x, c, w, z = data
        feature = x.numpy()
        # print("Feature Shape = ",np.shape(feature)[0])
        for j in range(np.shape(feature)[0]):
            cost = W_ols @ feature[j,:] + w0_ols
            sol_pred = solve_Shortest_Path(arcs_arr,cost)
            cost_pred = np.dot(sol_pred, c[j].to("cpu").detach().numpy())
            cost_pred_arr.append(cost_pred)
    # print("Average OLS Cost = ", np.mean(cost_pred_arr))
    return cost_pred_arr

In [22]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols, w0_ols, t_ols, obj_ols = ols_method_obj.ols_solver("",x_train, c_train)
cost_OLS_arr = evaluation_OLS(w0_ols,W_ols, loader_test)
print("Average OLS Cost = ",np.mean(cost_OLS_arr))

Average OLS Cost =  7.6799817386567595


# DDR

## Inner problem of the shortest path problem

In [23]:
def solve_Inner_Problem(arcs,cost_true,cost_pred,mu,is_binary):
    """
    A method to build Gurobi model

    Returns:
        tuple: optimization model and variables
    """
    import gurobipy as gp
    from gurobipy import GRB
    # import gurobipy as grb
    # ceate a model
    m = gp.Model("shortest path")
    m.setParam('OutputFlag', 0)
    if is_binary:
    # varibles
        x = m.addVars(arcs, lb = 0,vtype=GRB.BINARY  ,name="x")
    else:
        x = m.addVars(arcs, lb = 0,name="x")
    # sense
    # m.modelSense = GRB.MINIMIZE
    # flow conservation constraints
    for i in range(grid[0]):
        for j in range(grid[1]):
            v = i * grid[1] + j
            expr = 0
            for e in arcs:
                # flow in
                if v == e[1]:
                    expr += x[e]
                # flow out
                elif v == e[0]:
                    expr -= x[e]
            # source
            if i == 0 and j == 0:
                m.addConstr(expr == -1)
            # sink
            elif i == grid[0] - 1 and j == grid[0] - 1:
                m.addConstr(expr == 1)
            # transition
            else:
                m.addConstr(expr == 0)
    m.setObjective( sum([-mu*cost_true[ind] * x[arcs_arr[ind]] - (1-mu)*cost_pred[ind] * x[arcs_arr[ind]] for ind in range(len(arcs_arr))]) , GRB.MAXIMIZE)
    m.optimize()
    sol = m.getAttr('x')
    # print("sol = ",sol)
    shortest_path = obtain_path(arcs_arr,sol)
    obj = m.ObjVal
    # print("shortest_path = ",shortest_path)
    return obj,sol,shortest_path

## The dual problem of the inner problem

In [24]:
def solve_Inner_Problem_Dual(arcs_arr,cost_true,cost_pred,mu):
    num_nodes = 25
    import gurobipy as gp
    from gurobipy import GRB
    # import gurobipy as grb
    # ceate a model
    m = gp.Model("Maximium path")
    m.setParam('OutputFlag', 0)
    alpha = m.addVars(num_nodes,name="alpha")
    # sense
    # m.modelSense = GRB.MINIMIZE
    # flow conservation constraints
    for ind in range(len(arcs_arr)):
        e = arcs_arr[ind]
        j = e[1]
        i = e[0]
        # print("j = ",j,", i = ",i, ", e = ",e)
        m.addConstr(alpha[j] - alpha[i] >= -mu*cost_true[ind] - (1-mu)*cost_pred[ind])

    m.setObjective( alpha[num_nodes-1] - alpha[0], GRB.MINIMIZE)
    m.optimize()
    # # print("sol = ",sol)
    obj = m.ObjVal
    # print("obj = ",obj)
    return obj

# Check the equivalence between shortest path, its linear relaxation and dual problem

In [25]:
cost_true_tem = np.random.uniform(0,1,40) 
cost_pred_tem = np.random.uniform(0,1,40) 
mu_fixed = 0.4
obj_continous,sol_continous,shortest_path_con = solve_Inner_Problem(arcs_arr,cost_true_tem,cost_pred_tem,mu_fixed,False)
print("obj_continous = ",obj_continous)
print("shortest_path_con = ",shortest_path_con)

obj_binary,sol_binary,shortest_path_binary = solve_Inner_Problem(arcs_arr,cost_true_tem,cost_pred_tem,mu_fixed,True)
print("obj_binary = ",obj_binary)
print("shortest_path_binary = ",shortest_path_binary)


obj_dual = solve_Inner_Problem_Dual(arcs_arr,cost_true_tem,cost_pred_tem,mu_fixed)
print("obj_dual = ",obj_dual)


obj_continous =  -2.700267559919159
shortest_path_con =  [(0, 1), (1, 6), (6, 7), (7, 8), (8, 13), (13, 18), (18, 23), (23, 24)]
obj_binary =  -2.700267559919159
shortest_path_binary =  [(0, 1), (1, 6), (6, 7), (7, 8), (8, 13), (13, 18), (18, 23), (23, 24)]
obj_dual =  -2.7002675599191592


## Solve DDR

In [26]:
from gurobipy import *
def solve_DDR(lamb,mu_fixed,num_nodes,x_train,c_train):
    N,p = x_train.shape
    N,d = c_train.shape
    # print("Num of data = ",N, ",num of feature = ", p, ", num of acrs = ", d)
    
    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)])

    W_ddr = m.addVars( W_ind, lb=-GRB.INFINITY,name = "W" )
    w0_ddr = m.addVars( w0_ind, lb=-GRB.INFINITY,name = "W0" )
    alpha = m.addVars(N,num_nodes,name="alpha")
    expr_obj = 0
    err = []
    for n in range(N):
        cost_true_tem = c_train[n]
        expr_obj = expr_obj + alpha[n,num_nodes-1] - alpha[n,0]
        for ind in range(len(arcs_arr)):
            e = arcs_arr[ind]
            j = e[1]
            i = e[0]
            cost_pred_tem = quicksum([W_ddr[ind,j] * x_train[n,j] for j in range(p)]) + w0_ddr[ind]
            # print("j = ",j,", i = ",i, ", e = ",e)
            m.addConstr(alpha[n,j] - alpha[n,i] >= -mu_fixed*cost_true_tem[ind] - (1-mu_fixed)*cost_pred_tem)
            err.append(cost_true_tem[ind] - cost_pred_tem)
    m.setObjective(quicksum([err[k] * err[k] for k in range(len(err))])/N + lamb*(expr_obj)/N, GRB.MINIMIZE)
    m.optimize()
    W_DDR_rst = m.getAttr('x', W_ddr)
    w0_DDR_rst = m.getAttr('x', w0_ddr)
    W_ddr_val = []
    for i in range(d):
        W_ddr_val.append([W_DDR_rst[(i,j)] for j in range(p)])
    w0_ddr_val = [w0_DDR_rst[i] for i in range(d)]
    # print("w0_DDR_rst = ",w0_ddr_val)
    return w0_ddr_val,W_ddr_val 

In [27]:
print("Average True Cost = ",np.mean(cost_true_arr),"Std = ", np.std(cost_true_arr))
print("Average SPO Cost = ", np.mean(cost_pred_arr),"Std = ", np.std(cost_pred_arr))
print("Average OLS Cost = ", np.mean(cost_OLS_arr),"Std = ", np.std(cost_OLS_arr))

Average True Cost =  6.6683534908294675 Std =  1.2185374400285396
Average SPO Cost =  7.721849595680833 Std =  1.560122179102593
Average OLS Cost =  7.6799817386567595 Std =  1.5318926993671838


In [29]:
mu_arr = np.arange(0.1,1,0.05)
# lamb_arr = np.arange(0.05,0.2,0.05)
lamb_arr = np.arange(0.1,1,0.05)
# lamb_arr = [0.1]
minimum_value = 1000000000
for lamb in lamb_arr:
    print("======== lambda = ",lamb,"============")
    for mu in mu_arr:
        num_nodes = 25
        w0_ddr_val,W_ddr_val = solve_DDR(lamb,mu,num_nodes,x_train,c_train)
        cost_DDR_arr = evaluation_OLS(w0_ddr_val,W_ddr_val, loader_test)
        
        if np.mean(cost_DDR_arr) < minimum_value:
            minimum_value = np.mean(cost_DDR_arr)
            print("lambda = ",lamb, ", mu = ",mu, ", Lowerst Average DDR Cost = ",minimum_value, " Std = ",np.std(cost_DDR_arr))
        # print("lambda = ",lamb, ", mu = ",mu, ",Average DDR Cost = ",np.mean(cost_DDR_arr), " Std = ",np.std(cost_DDR_arr))

======== lambda =  0.1 ============
lambda =  0.1 , mu =  0.1 , Lowerst Average DDR Cost =  7.69086926355958  Std =  1.5374735989970505
lambda =  0.1 , mu =  0.15000000000000002 , Lowerst Average DDR Cost =  7.6902248392403125  Std =  1.5374344489144305
lambda =  0.1 , mu =  0.20000000000000004 , Lowerst Average DDR Cost =  7.683062559485435  Std =  1.5288134090938752
lambda =  0.1 , mu =  0.3500000000000001 , Lowerst Average DDR Cost =  7.680836561828852  Std =  1.5359545452277896
lambda =  0.1 , mu =  0.6000000000000002 , Lowerst Average DDR Cost =  7.68034748211503  Std =  1.5321031530001343
lambda =  0.1 , mu =  0.9000000000000002 , Lowerst Average DDR Cost =  7.677918752789497  Std =  1.5298038000216558
lambda =  0.1 , mu =  0.9500000000000003 , Lowerst Average DDR Cost =  7.676757207006216  Std =  1.5304983332245425
======== lambda =  0.15000000000000002 ============
======== lambda =  0.20000000000000004 ============
======== lambda =  0.25000000000000006 ============
======== l